In [ ]:
! pip install -U langchain_community langchain-openai langchain-anthropic langchain langgraph bs4

In [2]:

import getpass
import os


def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")


_set_env("OPENAI_API_KEY")
# _set_env("ANTHROPIC_API_KEY")

In [3]:
from bs4 import BeautifulSoup as Soup
from langchain_community.document_loaders.recursive_url_loader import RecursiveUrlLoader

# LCEL docs
url = "https://python.langchain.com/docs/concepts/#langchain-expression-language-lcel"
loader = RecursiveUrlLoader(
    url=url, max_depth=20, extractor=lambda x: Soup(x, "html.parser").text
)
docs = loader.load()

# Sort the list based on the URLs and get the text
d_sorted = sorted(docs, key=lambda x: x.metadata["source"])
d_reversed = list(reversed(d_sorted))
concatenated_content = "\n\n\n --- \n\n\n".join(
    [doc.page_content for doc in d_reversed]
)

In [6]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field

### OpenAI

# Grader prompt
code_gen_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """You are a coding assistant with expertise in LCEL, LangChain expression language. \n 
    Here is a full set of LCEL documentation:  \n ------- \n  {context} \n ------- \n Answer the user 
    question based on the above provided documentation. Ensure any code you provide can be executed \n 
    with all required imports and variables defined. Structure your answer with a description of the code solution. \n
    Then list the imports. And finally list the functioning code block. Here is the user question:""",
        ),
        ("placeholder", "{messages}"),
    ]
)


# Data model
class code(BaseModel):
    """Schema for code solutions to questions about LCEL."""

    prefix: str = Field(description="Description of the problem and approach")
    imports: str = Field(description="Code block import statements")
    code: str = Field(description="Code block not including import statements")


expt_llm = "gpt-4o-mini"
llm = ChatOpenAI(temperature=0, model=expt_llm)
code_gen_chain_oai = code_gen_prompt | llm.with_structured_output(code)
question = "How do I build a RAG chain in LCEL?"
solution = code_gen_chain_oai.invoke(
    {"context": concatenated_content, "messages": [("user", question)]}
)

code(prefix='To build a Retrieval-Augmented Generation (RAG) chain in LangChain Expression Language (LCEL), you will need to set up a chain that combines a retriever and a language model (LLM). The retriever will fetch relevant documents based on a query, and the LLM will generate responses based on those documents. Here’s how you can do it:', imports='from langchain_core.prompts import ChatPromptTemplate\nfrom langchain_openai import ChatOpenAI\nfrom langchain_core.output_parsers import StrOutputParser\nfrom langchain_core.retrievers import MyRetriever', code='# Define the retriever\nretriever = MyRetriever()  # Replace with your specific retriever implementation\n\n# Define the LLM\nllm = ChatOpenAI(model="gpt-4")\n\n# Create a prompt template for the LLM\nprompt_template = ChatPromptTemplate.from_template("Given the following documents, answer the question: {question}")\n\n# Build the RAG chain\nrag_chain = prompt_template | retriever | llm | StrOutputParser()\n\n# Example usage\nqu

In [8]:
from pprint import pprint

pprint(solution.prefix)
pprint(solution.imports)
pprint(solution.code)

('To build a Retrieval-Augmented Generation (RAG) chain in LangChain '
 'Expression Language (LCEL), you will need to set up a chain that combines a '
 'retriever and a language model (LLM). The retriever will fetch relevant '
 'documents based on a query, and the LLM will generate responses based on '
 'those documents. Here’s how you can do it:')
('from langchain_core.prompts import ChatPromptTemplate\n'
 'from langchain_openai import ChatOpenAI\n'
 'from langchain_core.output_parsers import StrOutputParser\n'
 'from langchain_core.retrievers import MyRetriever')
('# Define the retriever\n'
 'retriever = MyRetriever()  # Replace with your specific retriever '
 'implementation\n'
 '\n'
 '# Define the LLM\n'
 'llm = ChatOpenAI(model="gpt-4")\n'
 '\n'
 '# Create a prompt template for the LLM\n'
 'prompt_template = ChatPromptTemplate.from_template("Given the following '
 'documents, answer the question: {question}")\n'
 '\n'
 '# Build the RAG chain\n'
 'rag_chain = prompt_template | retr